# Sieve analysis

# Document information

<table align="left">
    <tr>
        <th class="text-align:left">Title</th>
        <td class="text-align:left">Sieve analysis</td>
    </tr>
    <tr>
        <th class="text-align:left">Last modified</th>
        <td class="text-align:left">2018-07-11</td>
    </tr>
    <tr>
        <th class="text-align:left">Author</th>
        <td class="text-align:left">Gilles Pilon <gillespilon13@gmail.com></td>
    </tr>
    <tr>
        <th class="text-align:left">Status</th>
        <td class="text-align:left">Active</td>
    </tr>
    <tr>
        <th class="text-align:left">Type</th>
        <td class="text-align:left">Jupyter notebook</td>
    </tr>
    <tr>
        <th class="text-align:left">Created</th>
        <td class="text-align:left">2018-07-11</td>
    </tr>
    <tr>
        <th class="text-align:left">File name</th>
        <td class="text-align:left">sieve_analysis.ipynb</td>
    </tr>
    <tr>
        <th class="text-align:left">Other files required</th>
        <td class="text-align:left">sieve_data.csv</td>
    </tr>
</table>

# In brevi

The purpose of this Jupyter notebook is to analyse sieve data.

# Methodology

TBD

# Data

Download the data file:

- TBD

# Import libraries and basic setup

In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.cm as cm
dots_c, line_c, *_ = cm.Paired.colors

# Read the data file

TBD

In [2]:
df = pd.read_csv('sieve_data.csv')

In [3]:
df.head()

,standard sieve,tyler sieve,sieve diameter,sieve mass,sieve soil mass,retained mass,retained percentage,passing percentage
0,NaN,3,6730.0,NaN,NaN,NaN,NaN,NaN
1,4,4,4760.0,NaN,NaN,NaN,NaN,NaN
2,5,5,4000.0,NaN,NaN,NaN,NaN,NaN
3,6,6,3360.0,NaN,NaN,NaN,NaN,NaN
4,7,7,2830.0,512.38,648.18,NaN,NaN,NaN


In [4]:
df.tail()

,standard sieve,tyler sieve,sieve diameter,sieve mass,sieve soil mass,retained mass,retained percentage,passing percentage
25,200,200,74.0,340.89,341.00,NaN,NaN,NaN
26,230,250,62.0,NaN,NaN,NaN,NaN,NaN
27,270,270,53.0,NaN,NaN,NaN,NaN,NaN
28,325,325,44.0,NaN,NaN,NaN,NaN,NaN
29,pan,pan,NaN,374.07,378.97,NaN,NaN,NaN


# Perform the analysis

In [5]:
df['retained mass'] = df['sieve soil mass'] - df['sieve mass']

In [6]:
df.dropna(subset=['sieve mass'], inplace=True)
df

,standard sieve,tyler sieve,sieve diameter,sieve mass,sieve soil mass,retained mass,retained percentage,passing percentage
4,7,7,2830.0,512.38,648.18,135.80,NaN,NaN
6,10,9,2000.0,464.38,653.00,188.62,NaN,NaN
10,18,16,1000.0,411.99,411.99,0.00,NaN,NaN
15,40,35,420.0,368.57,536.00,167.43,NaN,NaN
18,60,60,250.0,528.66,529.00,0.34,NaN,NaN
25,200,200,74.0,340.89,341.00,0.11,NaN,NaN
29,pan,pan,NaN,374.07,378.97,4.90,NaN,NaN


In [7]:
retained_mass_total = df['retained mass'].sum()
retained_mass_total

497.20000000000005

In [8]:
df['retained percentage'] = df['retained mass'] / retained_mass_total * 100

In [9]:
df

,standard sieve,tyler sieve,sieve diameter,sieve mass,sieve soil mass,retained mass,retained percentage,passing percentage
4,7,7,2830.0,512.38,648.18,135.80,27.312953,NaN
6,10,9,2000.0,464.38,653.00,188.62,37.936444,NaN
10,18,16,1000.0,411.99,411.99,0.00,0.000000,NaN
15,40,35,420.0,368.57,536.00,167.43,33.674578,NaN
18,60,60,250.0,528.66,529.00,0.34,0.068383,NaN
25,200,200,74.0,340.89,341.00,0.11,0.022124,NaN
29,pan,pan,NaN,374.07,378.97,4.90,0.985519,NaN


In [10]:
df['passing percentage'] = 100 - df['retained percentage']

In [11]:
df

,standard sieve,tyler sieve,sieve diameter,sieve mass,sieve soil mass,retained mass,retained percentage,passing percentage
4,7,7,2830.0,512.38,648.18,135.80,27.312953,72.687047
6,10,9,2000.0,464.38,653.00,188.62,37.936444,62.063556
10,18,16,1000.0,411.99,411.99,0.00,0.000000,100.000000
15,40,35,420.0,368.57,536.00,167.43,33.674578,66.325422
18,60,60,250.0,528.66,529.00,0.34,0.068383,99.931617
25,200,200,74.0,340.89,341.00,0.11,0.022124,99.977876
29,pan,pan,NaN,374.07,378.97,4.90,0.985519,99.014481
